In [3]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import special, integrate, stats, interpolate, spatial, optimize, signal
import pickle
import pandas as pd

import tqdm
import matplotlib as mpl
%config InlineBackend.figure_format = 'retina'
mpl.rcParams['figure.dpi']= 150
from astroquery.vizier import Vizier
import astropy.units as u

from hashlib import md5

def StanModel_cache(model_code, model_name=None, **kwargs):
    """Use just as you would `stan`"""
    code_hash = md5(model_code.encode('ascii')).hexdigest()
    if model_name is None:
        cache_fn = 'cached-model-{}.pkl'.format(code_hash)
    else:
        cache_fn = 'cached-{}-{}.pkl'.format(model_name, code_hash)
    try:
        sm = pickle.load(open(cache_fn, 'rb'))
    except:
        sm = pystan.StanModel(model_code=model_code)
        with open(cache_fn, 'wb') as f:
            pickle.dump(sm, f)
    else:
        print("Using cached StanModel")
    return sm

from matplotlib.patches import Ellipse
from matplotlib.lines import Line2D

### Set constants

In [2]:
gaia_mode = 'Gaia EDR3' # What do you want to query?
n_orbits = 100 # How many orbits do you want to integrate?
prior_lengthscale = 1.4 # What is the lengthscale of your distance prior? (kpc)
radial_velocity_prior = 0.0 # What is the mean of your radial velocity prior? (km/s)
radial_velocity_error_prior = 250.0 # What is the spread of your radial velocity prior? (km/s)

if gaia_mode == 'Gaia DR2':
    archive_key = 'gaiadr2'
    gaia_keys = ['ra','ra_error','dec','dec_error','parallax','parallax_error','pmra','pmra_error','pmdec','pmdec_error','ra_dec_corr','ra_parallax_corr','ra_pmra_corr','ra_pmdec_corr','dec_parallax_corr','dec_pmra_corr','dec_pmdec_corr','parallax_pmra_corr','parallax_pmdec_corr','pmra_pmdec_corr']
elif gaia_mode == 'Gaia EDR3':
    archive_key = 'gaiaedr3'
    gaia_keys = ['ra','ra_error','dec','dec_error','parallax','parallax_error','pmra','pmra_error','pmdec','pmdec_error','ra_dec_corr','ra_parallax_corr','ra_pmra_corr','ra_pmdec_corr','dec_parallax_corr','dec_pmra_corr','dec_pmdec_corr','parallax_pmra_corr','parallax_pmdec_corr','pmra_pmdec_corr','phot_g_mean_mag', 'nu_eff_used_in_astrometry', 'pseudocolour', 'ecl_lat', 'astrometric_params_solved','ruwe']

### Load data from file

In [4]:
data = pd.read_csv('./candidates.dat')
box = {ID:{'gaia_id':str(ID),'alias':data['ALIAS'][i],'radial_velocity':data['VRAD'][i],'radial_velocity_error':data['VRAD_ERR'][i]} for i,ID in enumerate(data['ID'])}
del data

In [5]:
for star in tqdm.tqdm_notebook(box.keys()):
    print(box[star]['alias'])

/Users/douglasboubert/anaconda3/envs/gaiaedr3/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


S5-HVS1
D6-1
D6-2
D6-3



# Query Archive

In [6]:
from astroquery.gaia import Gaia

for star in tqdm.notebook.tqdm(box.keys()):
    print(f"select * from {archive_key}.gaia_source where source_id={box[star]['gaia_id']}")
    job = Gaia.launch_job(f"select * from {archive_key}.gaia_source where source_id={box[star]['gaia_id']}")
    result = job.get_results()
    for gaia_key in gaia_keys:
        box[star][gaia_key] = result[gaia_key][0]

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


select * from gaiaedr3.gaia_source where source_id=6513109241989477504
select * from gaiaedr3.gaia_source where source_id=5805243926609660032
select * from gaiaedr3.gaia_source where source_id=1798008584396457088
select * from gaiaedr3.gaia_source where source_id=2156908318076164224



# Pre-processing

### Apply the Gaia recommend parallax zeropoint

In [7]:
if gaia_mode == 'Gaia DR2':
    for star in box.keys():
        box[star]['parallax_zeropoint'] = -0.029
        box[star]['parallax'] = box[star]['parallax'] - box[star]['parallax_zeropoint']
elif gaia_mode == 'Gaia EDR3':
    from zero_point import zpt
    zpt.load_tables()

    for star in box.keys():
        box[star]['parallax_zeropoint'] = zpt.get_zpt(box[star]['phot_g_mean_mag'], box[star]['nu_eff_used_in_astrometry'], box[star]['pseudocolour'], box[star]['ecl_lat'], box[star]['astrometric_params_solved'])
        box[star]['parallax'] = box[star]['parallax'] - box[star]['parallax_zeropoint'] 

/Users/douglasboubert/anaconda3/envs/gaiaedr3/lib/python3.7/site-packages/zero_point/zpt.py:163: UserWarning: Warning: converting a masked element to nan.
  pseudocolour = np.array([pseudocolour])


### Use a prior for stars without radial velocity information

In [8]:
for star in box.keys():
    if box[star]['radial_velocity_error'] < 0.0:
        box[star]['radial_velocity'] = radial_velocity_prior 
        box[star]['radial_velocity_error'] = radial_velocity_error_prior 

# Create objects for sampling   

In [9]:
gaia_root_keys = ['ra','dec','parallax','pmra','pmdec']
for star in box.keys():
    box[star]['corr'] = np.ones((5,5))
    for i,I in enumerate(gaia_root_keys):
        for j,J in enumerate(gaia_root_keys):
            if i < j:
                box[star]['corr'][i,j] = box[star]['corr'][j,i] = box[star][I+'_'+J+'_corr']
    
    box[star]['mean'] = np.array([box[star][k] for k in gaia_root_keys])
    box[star]['mean'][:2] = 0.0
    box[star]['error'] = np.array([box[star][k+'_error'] for k in gaia_root_keys])
    
    box[star]['cov'] = np.array([[box[star]['corr'][i,j]*box[star]['error'][i]*box[star]['error'][j] for j in range(5)] for i in range(5)])

# Sampling

### Compile the Stan sampling (this will take two minutes the first time you run it!)

In [11]:
import pystan

stan_code = """
data {
  vector[5] mu;
  matrix[5,5] Sigma;
  real lengthscale;
}
parameters {
  real ra;
  real dec;
  real dist;
  real pmra;
  real pmdec;
}
model {
  dist ~ gamma(3.0, 1.0/lengthscale);
  ra ~ normal(0,10);
  dec ~ normal(0,10);
  pmra ~ normal(0,10);
  pmdec ~ normal(0,10);
  mu ~ multi_normal([ra, dec, 1.0/dist,pmra,pmdec]',Sigma);
}
"""
sm = StanModel_cache(model_code=stan_code,model_name='bailer-jones')

Using cached StanModel


### Sample for each star

In [12]:
for star in tqdm.notebook.tqdm(box.keys()):
    stan_data = {'mu':box[star]['mean'],'Sigma':box[star]['cov'],'lengthscale':prior_lengthscale}
    fit = sm.sampling(data=stan_data, iter=20000, chains=4)
    
    stan_result = fit.extract()
    box[star]['samples'] = {'ra':stan_result['ra'],'dec':stan_result['dec'],'distance':stan_result['dist'],'pmra':stan_result['pmra'],'pmdec':stan_result['pmdec']}
    box[star]['samples']['ra'] += box[star]['ra']
    box[star]['samples']['dec'] += box[star]['dec']
    box[star]['samples']['radial_velocity'] = np.random.normal(box[star]['radial_velocity'],box[star]['radial_velocity_error'],box[star]['samples']['ra'].size)

# Orbit integration

In [14]:
# Third-party
import astropy.coordinates as coord
from astropy.io import ascii

# Gala
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

coord.galactocentric_frame_defaults.set('v4.0')
potential = gp.MilkyWayPotential()
gc_frame = coord.Galactocentric()

In [15]:
for star in tqdm.notebook.tqdm(box.keys()):
    icrs_samples = coord.SkyCoord(ra=box[star]['samples']['ra']*u.deg,
                                  dec=box[star]['samples']['dec']*u.deg,
                                  distance=box[star]['samples']['distance']*u.kpc,
                                  pm_ra_cosdec=box[star]['samples']['pmra']*u.mas/u.yr,
                                  pm_dec=box[star]['samples']['pmdec']*u.mas/u.yr,
                                  radial_velocity=box[star]['samples']['radial_velocity']*u.km/u.s)
    
    galcen_samples = icrs_samples.transform_to(gc_frame)
    box[star]['samples']['total_velocity'] = np.sqrt(galcen_samples.v_x.value**2.0+galcen_samples.v_y.value**2.0+galcen_samples.v_z.value**2.0)
    w0_samples = gd.PhaseSpacePosition(galcen_samples.data)
    samples = np.random.choice(np.arange(box[star]['samples']['ra'].size),n_orbits)

    orbit_backward_samples = potential.integrate_orbit(w0_samples[samples], dt=-0.1*u.Myr, n_steps=10000)
    box[star]['samples']['orbit_backward_x'] = orbit_backward_samples.x.value
    box[star]['samples']['orbit_backward_y'] = orbit_backward_samples.y.value
    box[star]['samples']['orbit_backward_z'] = orbit_backward_samples.z.value

    orbit_forward_samples = potential.integrate_orbit(w0_samples[samples], dt=+0.1*u.Myr, n_steps=10000)
    box[star]['samples']['orbit_forward_x'] = orbit_forward_samples.x.value#[1:]
    box[star]['samples']['orbit_forward_y'] = orbit_forward_samples.y.value#[1:]
    box[star]['samples']['orbit_forward_z'] = orbit_forward_samples.z.value#[1:]

### Posterior total velocity

In [16]:
for star in box.keys():
    perc = np.percentile(box[star]['samples']['total_velocity'],[16,50,84])
    print(box[star]['alias'],perc[0],0.5*(perc[2]-perc[0]))

S5-HVS1 1490.7522570858396 296.7259536573829
D6-1 1811.942319031251 208.417798015073
D6-2 1068.5789461761858 55.44459176268583
D6-3 2011.4801045696322 663.9851432732703


# Orbit Plot

In [ ]:
label_fontsize = 12
marker_size = 80
line_alpha = 0.05
fig,axes = plt.subplots(1,2,figsize=(9,5))

stars_to_plot = [6513109241989477504,5805243926609660032,1798008584396457088,2156908318076164224]

for star in tqdm.notebook.tqdm(stars_to_plot):
    axes[0].plot(box[star]['samples']['orbit_backward_x'],box[star]['samples']['orbit_backward_y'],color=plt.cm.RdBu(0.8),alpha=line_alpha, solid_capstyle='butt');
    axes[0].plot(box[star]['samples']['orbit_forward_x'],box[star]['samples']['orbit_forward_y'],color=plt.cm.RdBu(0.2),alpha=line_alpha, solid_capstyle='butt');
    axes[0].set_xlim([-25,+25])
    axes[0].set_ylim([-25,+25])
    axes[0].set_aspect('equal')
    axes[0].set_xlabel(r'$x\;(\mathrm{kpc})$',fontsize=label_fontsize)
    axes[0].set_ylabel(r'$y\;(\mathrm{kpc})$',fontsize=label_fontsize,labelpad=-5)
    axes[0].scatter(-gc_frame.galcen_distance.value,0,marker='*',facecolor='gold',edgecolor='black', linewidth=0.4,zorder=10,s=marker_size)
    axes[0].scatter(0,0,marker='+',facecolor='black',edgecolor='black', linewidth=1.5,zorder=10,s=marker_size)
    disk_top = Ellipse(xy=(0, 0), width=30.0, height=30.0, edgecolor='grey', fc='None', lw=2,ls='--',zorder=10)
    axes[0].add_patch(disk_top)

    axes[1].plot(box[star]['samples']['orbit_backward_x'],box[star]['samples']['orbit_backward_z'],color=plt.cm.RdBu(0.8),alpha=line_alpha, solid_capstyle='butt');
    axes[1].plot(box[star]['samples']['orbit_forward_x'],box[star]['samples']['orbit_forward_z'],color=plt.cm.RdBu(0.2),alpha=line_alpha, solid_capstyle='butt');
    axes[1].set_xlim([-25,+25])
    axes[1].set_ylim([-25,+25])
    axes[1].set_aspect('equal')
    axes[1].set_xlabel(r'$x\;(\mathrm{kpc})$',fontsize=label_fontsize)
    axes[1].set_ylabel(r'$z\;(\mathrm{kpc})$',fontsize=label_fontsize,labelpad=-5)
    axes[1].scatter(-gc_frame.galcen_distance.value,0,marker='*',facecolor='gold',edgecolor='black', linewidth=0.4,zorder=10,s=marker_size)
    axes[1].scatter(0,0,marker='+',facecolor='black',edgecolor='black', linewidth=1.5,zorder=10,s=marker_size)
    disk_side = Ellipse(xy=(0, 0), width=30.0, height=3.0, edgecolor='grey', fc='None', lw=2,ls='--',zorder=10)
    axes[1].add_patch(disk_side)

    
    custom_lines = [Line2D([0], [0], color=plt.cm.RdBu(0.8), lw=4),
                    Line2D([0], [0], color=plt.cm.RdBu(0.2), lw=4)]

    axes[1].legend(custom_lines, [box[star]['alias']+' past', box[star]['alias']+' future'],loc='lower right',frameon=False,fontsize=label_fontsize)
    plt.savefig('./results/'+box[star]['alias']+'_orbit.png',dpi=300,bbox_inches='tight',facecolor='white')
    plt.savefig('./results/'+box[star]['alias']+'_orbit.pdf',dpi=300,bbox_inches='tight')
    
    axes[0].clear()
    axes[1].clear()

# Velocity plot

In [ ]:
label_fontsize = 12

plt.figure(figsize=(6,4))
stars_to_plot = [6513109241989477504,5805243926609660032,1798008584396457088,2156908318076164224]
color_to_plot = [plt.cm.RdBu(0.8),plt.cm.RdBu(0.2),plt.cm.PiYG(0.8),plt.cm.PiYG(0.2)]
custom_lines = [Line2D([0], [0], color=color, lw=4) for color in color_to_plot]
custom_label = [box[star]['alias'] for star in stars_to_plot]


for star in zip(tqdm.notebook.tqdm(stars_to_plot),color_to_plot):
    x = np.linspace(0,5000,1000)
    y = stats.gaussian_kde(box[star]['samples']['total_velocity'],bw_method=0.2)(x)
    plt.plot(x,y,color=color,label=box[star]['alias'],lw=2)
plt.xlim([0,5000])
plt.ylim([0,0.0075])
plt.legend(custom_lines,custom_label,loc='upper right',frameon=False,fontsize=label_fontsize)
plt.xlabel('Total Galactocentric Velocity ($\mathrm{km}\;\mathrm{s}^{-1}$)',fontsize=label_fontsize )
plt.ylabel('Posterior Probability Density ($1/\mathrm{km}\;\mathrm{s}^{-1}$)',fontsize=label_fontsize)

plt.savefig('./results/posterior_velocity.png',dpi=300,bbox_inches='tight')
plt.savefig('./results/posterior_velocity.pdf',dpi=300,bbox_inches='tight')